In [29]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import utils
import plotly.graph_objects as go
from instrument import Instrument

In [30]:
# pair = "EUR_USD"
# pair = "GBP_USD"
# pair = "USD_JPY"
pair = "GBP_JPY"
granularity = "H1"
ma_list = [16,64]
i_pair = Instrument.get_instrument_by_name(pair, True)

In [31]:
i_pair

{'name': 'GBP_JPY', 'ins_type': 'CURRENCY', 'displayName': 'GBP/JPY', 'pipLocation': 0.01, 'marginRate': '0.0333'}

In [32]:
df = pd.read_pickle(utils.get_data_filename(pair, granularity, True))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [33]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [34]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-12-21T19:00:00.000000000Z,180.407,180.464,180.297,180.402,180.528188,181.910547
1,2023-12-21T20:00:00.000000000Z,180.401,180.496,180.358,180.422,180.517687,181.892000
2,2023-12-21T21:00:00.000000000Z,180.422,180.479,180.314,180.350,180.488812,181.871688
3,2023-12-21T22:00:00.000000000Z,180.424,180.490,180.270,180.305,180.457750,181.847672
4,2023-12-21T23:00:00.000000000Z,180.313,180.385,180.102,180.136,180.402875,181.808781


In [35]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [36]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [37]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
31,2023-12-26T02:00:00.000000000Z,180.766,180.806,180.656,180.716,180.859625,180.852859,0.006766,-0.004141,1
85,2023-12-28T08:00:00.000000000Z,180.512,180.704,180.299,180.388,181.148250,181.152828,-0.004578,0.068078,-1
163,2024-01-03T14:00:00.000000000Z,180.631,181.055,180.552,180.930,179.904750,179.861937,0.042813,-0.044906,1
241,2024-01-08T20:00:00.000000000Z,183.864,183.924,183.782,183.832,183.550500,183.557750,-0.007250,0.002344,-1
274,2024-01-10T05:00:00.000000000Z,183.944,184.024,183.790,183.902,183.659188,183.623719,0.035469,-0.027187,1


In [38]:
df_trades['DELTA'] = ( df_trades.mid_c.diff() / i_pair.pipLocation ).shift(-1)
df_trades['GAIN'] = df_trades.DELTA * df_trades.IS_TRADE

In [39]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
31,2023-12-26T02:00:00.000000000Z,180.766,180.806,180.656,180.716,180.859625,180.852859,0.006766,-0.004141,1,-32.8,-32.8
85,2023-12-28T08:00:00.000000000Z,180.512,180.704,180.299,180.388,181.148250,181.152828,-0.004578,0.068078,-1,54.2,-54.2
163,2024-01-03T14:00:00.000000000Z,180.631,181.055,180.552,180.930,179.904750,179.861937,0.042813,-0.044906,1,290.2,290.2
241,2024-01-08T20:00:00.000000000Z,183.864,183.924,183.782,183.832,183.550500,183.557750,-0.007250,0.002344,-1,7.0,-7.0
274,2024-01-10T05:00:00.000000000Z,183.944,184.024,183.790,183.902,183.659188,183.623719,0.035469,-0.027187,1,77.4,77.4


In [40]:
df_trades['GAIN'].sum()

np.float64(1136.3999999999946)

In [45]:
df_plot = df_ma.iloc[150:300].copy()
df_plot.shape

(150, 10)

In [46]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()